## Partitioning

In [ ]:
%%sql
--Partitions 1 - List
CREATE TABLE sales_data_partition_1 (
    date TIMESTAMP,
    name VARCHAR(50),
    market_area VARCHAR(100),
    number_of_sales INTEGER,
    pricing_unit INTEGER
) PARTITION BY list(name);

--Partition 1 - manual partition define
CREATE TABLE diah PARTITION OF sales_data_partition_1
FOR VALUES IN ('Diah');
CREATE TABLE wahyu PARTITION OF sales_data_partition_1
FOR VALUES IN ('Wahyu');
CREATE TABLE lisa PARTITION OF sales_data_partition_1
FOR VALUES IN ('Lisa');
CREATE TABLE anton PARTITION OF sales_data_partition_1
FOR VALUES IN ('Anton');
CREATE TABLE malik PARTITION OF sales_data_partition_1
FOR VALUES IN ('Malik');
CREATE TABLE riana PARTITION OF sales_data_partition_1
FOR VALUES IN ('Riana');
CREATE TABLE rafi PARTITION OF sales_data_partition_1
FOR VALUES IN ('Rafi');
CREATE TABLE bela PARTITION OF sales_data_partition_1
FOR VALUES IN ('Bela');
CREATE TABLE budi PARTITION OF sales_data_partition_1
FOR VALUES IN ('Budi');

In [ ]:
%%sql

--Partition 2 - Range
CREATE TABLE sales_data_partition_2 (
    date TIMESTAMP,
    name VARCHAR(50),
    market_area VARCHAR(100),
    number_of_sales INTEGER,
    pricing_unit INTEGER
)PARTITION BY RANGE (date);


--Partition 2 - manual define
CREATE TABLE sales_january PARTITION OF sales_data_partition_2
    FOR VALUES FROM ('2023-01-01') TO ('2023-02-01');

CREATE TABLE sales_february PARTITION OF sales_data_partition_2
    FOR VALUES FROM ('2023-02-01') TO ('2023-03-01');

CREATE TABLE sales_march PARTITION OF sales_data_partition_2
    FOR VALUES FROM ('2023-03-01') TO ('2023-04-01');
---
---
---
--- so on

In [ ]:
%%sql

--Checking Partitions
SELECT
    inhrelid::regclass AS partition_name,
    inhparent::regclass AS parent_table,
    pg_size_pretty(pg_total_relation_size(inhrelid::regclass)) AS partition_size
FROM
    pg_inherits
WHERE
    inhparent = 'sales_data_partition_1'::regclass; --name of table

## Common Table Expression

In [ ]:
%%sql
--CTE with our current table
--Trial 1
WITH average_pricing AS (
  SELECT
    market_area,
    AVG(pricing_unit) AS average_price
  FROM
    sales_data_partition_1
  GROUP BY
    market_area
)
SELECT *
FROM average_pricing

--Trial 2
WITH sales_performance AS (
  SELECT
    name,
    SUM(number_of_sales * pricing_unit) AS total_revenue
  FROM
    sales_data_partition_1
  GROUP BY
    name
)
SELECT
  name,
  total_revenue
FROM
  sales_performance
ORDER BY
  total_revenue DESC
LIMIT 5;

## Materialized View

In [ ]:
%%sql

--Creating MV
CREATE MATERIALIZED VIEW sales_aggegation_mv
AS
SELECT name, count(*)
FROM sales_data_partition_1
GROUP BY 1
ORDER BY 2 DESC;

--Function to refresh
CREATE OR REPLACE FUNCTION refresh_sales_aggegation_mv() RETURNS trigger AS $$
BEGIN
    REFRESH MATERIALIZED VIEW sales_aggegation_mv;
    RETURN NULL;
END;
$$ LANGUAGE plpgsql;

--Add function as trigger
CREATE TRIGGER refresh_sales_aggegation_mv_trigger
AFTER INSERT OR UPDATE OR DELETE ON sales_data_partition_1
FOR EACH STATEMENT EXECUTE PROCEDURE refresh_sales_aggegation_mv();